In [ ]:
import numpy as np

class ActivationFunctions:
    """Collection of activation functions and their derivatives"""
    
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    @staticmethod
    def sigmoid_derivative(y):
        return y * (1 - y)
    
    @staticmethod
    def relu(x):
        return np.maximum(0, x)
    
    @staticmethod
    def relu_derivative(x):
        return (x > 0).astype(float)
    
    @staticmethod
    def leaky_relu(x, alpha=0.01):
        return np.where(x > 0, x, alpha * x)
    
    @staticmethod
    def leaky_relu_derivative(x, alpha=0.01):
        return np.where(x > 0, 1, alpha)
    
    @staticmethod
    def tanh(x):
        return np.tanh(x)
    
    @staticmethod
    def tanh_derivative(x):
        return 1 - np.tanh(x)**2
    
    @staticmethod
    def swish(x):
        return x / (1 + np.exp(-np.clip(x, -500, 500)))
    
    @staticmethod
    def swish_derivative(x):
        sig = 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        return sig + x * sig * (1 - sig)
    
    @staticmethod
    def softmax(x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    @staticmethod
    def softmax_derivative(softmax_output):
        s = softmax_output.reshape(-1, 1)
        return np.diagflat(s) - np.dot(s, s.T)


class NeuralNetwork:
    """Simple feedforward neural network with backpropagation"""
    
    def __init__(self, learning_rate=0.71, error_threshold=0.001):
        self.learning_rate = learning_rate
        self.error_threshold = error_threshold
        self.activation = ActivationFunctions()
        
        # Initialize weights and biases
        self.W1 = np.array([
            [0.3, 0.1, -0.2],
            [-0.2, 0.4, 0.3],
            [0.2, -0.3, 0.1],
            [0.1, 0.4, -0.1]
        ])
        self.b1 = np.array([[0.2, 0.1, 0.05]])
        
        self.W2 = np.array([
            [0.3, -0.2],
            [0.1, 0.4],
            [-0.3, 0.2]
        ])
        self.b2 = np.array([[0.1, -0.2]])
        
        self.V = np.array([[0.2], [-0.3]])
        self.C = np.array([[0.1]])
        
        self.epoch_count = 0
        self.first_epoch_printed = False
    
    def forward(self, X):
        """Forward propagation through the network"""
        # First hidden layer
        self.z1 = np.dot(X, self.W1) + self.b1
        self.h1 = self.activation.relu(self.z1)
        
        # Second hidden layer
        self.z2 = np.dot(self.h1, self.W2) + self.b2
        self.h2 = self.activation.relu(self.z2)
        
        # Output layer
        self.u = np.dot(self.h2, self.V) + self.C
        self.output = self.activation.sigmoid(self.u)
        
        return self.output
    
    def backward(self, X, Y):
        """Backpropagation to compute gradients"""
        # Calculate error
        error = Y - self.output
        
        # Output layer gradients
        delta_output = error * self.activation.sigmoid_derivative(self.output)
        
        # Second hidden layer gradients
        delta_h2 = delta_output.dot(self.V.T) * self.activation.relu_derivative(self.z2)
        
        # First hidden layer gradients
        delta_h1 = delta_h2.dot(self.W2.T) * self.activation.relu_derivative(self.z1)
        
        # Update weights and biases
        self.V += self.learning_rate * self.h2.T.dot(delta_output)
        self.C += self.learning_rate * delta_output
        
        self.W2 += self.learning_rate * self.h1.T.dot(delta_h2)
        self.b2 += self.learning_rate * delta_h2
        
        self.W1 += self.learning_rate * X.T.dot(delta_h1)
        self.b1 += self.learning_rate * delta_h1
        
        return error
    
    def print_weights(self, prefix=""):
        """Print current weights and biases"""
        print(f"\n{prefix}W1:")
        for i in range(self.W1.shape[0]):
            for j in range(self.W1.shape[1]):
                print(f"  W1_{i+1}{j+1}: {self.W1[i, j]:.6f}")
        
        print(f"{prefix}b1:")
        for j in range(self.b1.shape[1]):
            print(f"  b1_{j+1}: {self.b1[0, j]:.6f}")
        
        print(f"{prefix}W2:")
        for i in range(self.W2.shape[0]):
            for j in range(self.W2.shape[1]):
                print(f"  W2_{i+1}{j+1}: {self.W2[i, j]:.6f}")
        
        print(f"{prefix}b2:")
        for j in range(self.b2.shape[1]):
            print(f"  b2_{j+1}: {self.b2[0, j]:.6f}")
        
        print(f"{prefix}V (Output weights):")
        for i in range(self.V.shape[0]):
            print(f"  V{i+1}: {self.V[i, 0]:.6f}")
        
        print(f"{prefix}C (Output bias):")
        print(f"  C: {self.C[0, 0]:.6f}")
    
    def train(self, X, Y, max_epochs=100000):
        """Train the neural network"""
        print("Training started...")
        
        while self.epoch_count < max_epochs:
            self.epoch_count += 1
            
            # Forward and backward pass
            output = self.forward(X)
            error = self.backward(X, Y)
            
            # Print first epoch details
            if not self.first_epoch_printed:
                print("\n" + "="*50)
                print("FIRST EPOCH")
                print("="*50)
                print(f"Output: {output.item():.6f}")
                print(f"Error: {error.item():.6f}")
                self.print_weights("Updated ")
                self.first_epoch_printed = True
            
            # Check convergence
            if abs(error.item()) < self.error_threshold:
                print("\n" + "="*50)
                print("TRAINING COMPLETED")
                print("="*50)
                print(f"Total Epochs: {self.epoch_count}")
                print(f"Final Output: {output.item():.6f}")
                print(f"Final Error: {error.item():.6f}")
                self.print_weights("Final ")
                break
        
        if self.epoch_count >= max_epochs:
            print(f"\nReached maximum epochs ({max_epochs}) without convergence")
    
    def predict(self, X):
        """Make predictions on new data"""
        return self.forward(X)


def main():
    """Main execution function"""
    # Training data
    X = np.array([[1, 1, 0, 1]])
    Y = np.array([[1]])
    
    # Create and train network
    nn = NeuralNetwork(learning_rate=0.71, error_threshold=0.001)
    nn.train(X, Y)
    
    # Test prediction
    print("\n" + "="*50)
    print("PREDICTION TEST")
    print("="*50)
    prediction = nn.predict(X)
    print(f"Input: {X[0]}")
    print(f"Predicted Output: {prediction.item():.6f}")
    print(f"Target Output: {Y.item()}")


# Avoid auto-running the demo when the file is loaded inside a notebook
if __name__ == "__main__" and 'get_ipython' not in globals():
    main()

Training started...

FIRST EPOCH
Output: 0.530462
Error: 0.469538

Updated W1:
  W1_11: 0.309964
  W1_12: 0.091697
  W1_13: -0.209964
  W1_21: -0.190036
  W1_22: 0.391697
  W1_23: 0.290036
  W1_31: 0.200000
  W1_32: -0.300000
  W1_33: 0.100000
  W1_41: 0.109964
  W1_42: 0.391697
  W1_43: -0.109964
Updated b1:
  b1_1: 0.209964
  b1_2: 0.091697
  b1_3: 0.040036
Updated W2:
  W2_11: 0.306643
  W2_12: -0.209964
  W2_21: 0.116607
  W2_22: 0.375090
  W2_31: -0.299170
  W2_32: 0.198754
Updated b2:
  b2_1: 0.116607
  b2_2: -0.224910
Updated V (Output weights):
  V1: 0.225325
  V2: -0.289206
Updated C (Output bias):
  C: 0.183034

TRAINING COMPLETED
Total Epochs: 17686
Final Output: 0.999000
Final Error: 0.001000

Final W1:
  W1_11: 0.632159
  W1_12: 0.346731
  W1_13: -0.219515
  W1_21: 0.132159
  W1_22: 0.646731
  W1_23: 0.280485
  W1_31: 0.200000
  W1_32: -0.300000
  W1_33: 0.100000
  W1_41: 0.432159
  W1_42: 0.646731
  W1_43: -0.119515
Final b1:
  b1_1: 0.532159
  b1_2: 0.346731
  b1_3: 0.03

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_curve, auc, mean_squared_error, r2_score


class LinearRegressionModel:
    """
    A simple linear regression model implemented from scratch using gradient descent.
    """
    
    def __init__(self, n_features):
        """
        Initialize the model with random weights and bias.
        
        Args:
            n_features: Number of input features
        """
        self.weights = np.random.randn(n_features, 1) * 0.01
        self.bias = np.random.randn(1)
        self.loss_history = []
    
    def predict(self, X):
        """
        Make predictions using the linear model: y = Xw + b
        
        Args:
            X: Input features (n_samples, n_features)
            
        Returns:
            Predictions (n_samples, 1)
        """
        return X.dot(self.weights) + self.bias
    
    def compute_loss(self, y_true, y_pred):
        """
        Compute Mean Squared Error loss.
        
        Args:
            y_true: True target values
            y_pred: Predicted values
            
        Returns:
            MSE loss value
        """
        return np.mean((y_true - y_pred) ** 2)
    
    def train(self, X, y, learning_rate=0.01, epochs=200, verbose=True):
        """
        Train the model using gradient descent.
        
        Args:
            X: Training features (n_samples, n_features)
            y: Target values (n_samples, 1)
            learning_rate: Step size for gradient descent
            epochs: Number of training iterations
            verbose: Whether to print training progress
            
        Returns:
            List of loss values for each epoch
        """
        n_samples = len(y)
        
        for epoch in range(epochs):
            # Forward pass
            y_pred = self.predict(X)
            
            # Compute loss
            loss = self.compute_loss(y, y_pred)
            self.loss_history.append(loss)
            
            # Compute gradients
            error = y - y_pred
            grad_weights = -2 * X.T.dot(error) / n_samples
            grad_bias = -2 * np.mean(error)
            
            # Update parameters
            self.weights -= learning_rate * grad_weights
            self.bias -= learning_rate * grad_bias
            
            # Print progress
            if verbose and (epoch + 1) % 50 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")
        
        return self.loss_history


class HousingPricePredictor:
    """
    Complete pipeline for housing price prediction including data preprocessing,
    model training, and visualization.
    """
    
    def __init__(self, csv_path_or_df):
        """
        Initialize the predictor with data from a CSV file or a pandas.DataFrame.

        Args:
            csv_path_or_df: Path to the housing dataset CSV file or a pandas.DataFrame
        """
        import os
        if isinstance(csv_path_or_df, pd.DataFrame):
            # accept DataFrame directly (useful for notebooks/tests)
            self.data = csv_path_or_df.copy()
        else:
            # validate path early and give a helpful error message
            if not os.path.exists(csv_path_or_df):
                raise FileNotFoundError(
                    f"{csv_path_or_df!r} not found. Provide a valid CSV path or a pandas.DataFrame."
                )
            self.data = pd.read_csv(csv_path_or_df)

        self.model = None
        self.scaler = StandardScaler()
        self.X_scaled = None
        self.y = None
        
    def preprocess_data(self):
        """
        Preprocess the data: encode categorical variables and scale features.
        """
        print("Preprocessing data...")
        
        # Categorical columns to encode
        categorical_columns = [
            'mainroad', 'guestroom', 'basement', 'hotwaterheating',
            'airconditioning', 'prefarea', 'furnishingstatus'
        ]
        
        # Encode categorical variables
        for col in categorical_columns:
            label_encoder = LabelEncoder()
            self.data[col] = label_encoder.fit_transform(self.data[col])
        
        # Separate features and target
        self.X = self.data.drop("price", axis=1).values.astype(float)
        self.y = self.data["price"].values.astype(float).reshape(-1, 1)
        
        # Scale features
        self.X_scaled = self.scaler.fit_transform(self.X)
        
        print(f"Data shape: {self.X_scaled.shape}")
        print(f"Target shape: {self.y.shape}")
    
    def train_model(self, learning_rate=0.01, epochs=200):
        """
        Train the linear regression model.
        
        Args:
            learning_rate: Learning rate for gradient descent
            epochs: Number of training epochs
        """
        print(f"\nTraining model for {epochs} epochs...")
        
        self.model = LinearRegressionModel(n_features=self.X_scaled.shape[1])
        self.model.train(
            self.X_scaled, 
            self.y, 
            learning_rate=learning_rate, 
            epochs=epochs,
            verbose=True
        )
        
        print("Training completed!")
    
    def evaluate_model(self):
        """
        Evaluate the model and print performance metrics.
        """
        y_pred = self.model.predict(self.X_scaled)
        
        mse = mean_squared_error(self.y, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(self.y, y_pred)
        
        print("\n" + "="*50)
        print("MODEL EVALUATION")
        print("="*50)
        print(f"Mean Squared Error: {mse:,.2f}")
        print(f"Root Mean Squared Error: {rmse:,.2f}")
        print(f"R² Score: {r2:.4f}")
        print("="*50)
    
    def plot_predictions(self):
        """
        Plot predicted vs actual values with best fit line.
        """
        y_pred = self.model.predict(self.X_scaled)
        
        plt.figure(figsize=(10, 6))
        plt.scatter(self.y, y_pred, color='blue', alpha=0.5, label='Predictions')
        
        # Calculate and plot best fit line
        coeffs = np.polyfit(self.y.ravel(), y_pred.ravel(), 1)
        best_fit_line = coeffs[0] * self.y + coeffs[1]
        plt.plot(self.y, best_fit_line, color='red', linewidth=2, label='Best Fit Line')
        
        # Perfect prediction line
        plt.plot(self.y, self.y, color='green', linestyle='--', linewidth=1, label='Perfect Prediction')
        
        plt.xlabel("Actual Price", fontsize=12)
        plt.ylabel("Predicted Price", fontsize=12)
        plt.title("Housing Price Prediction: Actual vs Predicted", fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def plot_roc_curve(self):
        """
        Plot ROC curve for binary classification (above/below median price).
        """
        # Convert to binary classification problem
        median_price = np.median(self.y)
        y_binary = (self.y >= median_price).astype(int).ravel()
        y_scores = self.model.predict(self.X_scaled).ravel()
        
        # Compute ROC curve and AUC
        fpr, tpr, thresholds = roc_curve(y_binary, y_scores)
        roc_auc = auc(fpr, tpr)
        
        # Plot
        plt.figure(figsize=(10, 6))
        plt.plot(fpr, tpr, color='darkorange', linewidth=2, 
                label=f'ROC Curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', linewidth=2, linestyle='--', 
                label='Random Classifier')
        
        plt.xlabel("False Positive Rate", fontsize=12)
        plt.ylabel("True Positive Rate", fontsize=12)
        plt.title("ROC Curve - Binary Classification (Above/Below Median Price)", 
                 fontsize=14, fontweight='bold')
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print(f"\nROC AUC Score: {roc_auc:.4f}")
    
    def plot_loss_curve(self):
        """
        Plot the training loss over epochs.
        """
        plt.figure(figsize=(10, 6))
        plt.plot(self.model.loss_history, color='blue', linewidth=2)
        plt.xlabel("Epoch", fontsize=12)
        plt.ylabel("Mean Squared Error", fontsize=12)
        plt.title("Training Loss Over Epochs", fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def run_complete_pipeline(self, learning_rate=0.01, epochs=200):
        """
        Run the complete prediction pipeline from preprocessing to visualization.
        
        Args:
            learning_rate: Learning rate for model training
            epochs: Number of training epochs
        """
        print("="*50)
        print("HOUSING PRICE PREDICTION PIPELINE")
        print("="*50)
        
        # Step 1: Preprocess data
        self.preprocess_data()
        
        # Step 2: Train model
        self.train_model(learning_rate=learning_rate, epochs=epochs)
        
        # Step 3: Evaluate model
def main(housing_csv="Housing.csv"):
        
    Main execution function. Pass a path or a pandas.DataFrame.
        print("\nGenerating visualizations...")
    import os
    if not os.path.exists(housing_csv):
        raise FileNotFoundError(
            f"{housing_csv!r} not found. To run the demo place the CSV in the notebook folder "
            "or call HousingPricePredictor with a pandas.DataFrame."
        )

    # Initialize predictor
    predictor = HousingPricePredictor(housing_csv)


    Main execution function.    main()

    # Run complete pipeline
    """if __name__ == "__main__":

    predictor.run_complete_pipeline(learning_rate=0.01, epochs=200)
    # Initialize predictor


    predictor = HousingPricePredictor("Housing.csv")


        main()

# Prevent auto-execution inside interactive notebook kernels
    # Run complete pipeline    predictor.run_complete_pipeline(learning_rate=0.01, epochs=200)
if __name__ == "__main__" and 'get_ipython' not in globals():

SyntaxError: unterminated triple-quoted string literal (detected at line 304) (2905519407.py, line 291)